# Explore United States Immigration Data
### Data Engineering Capstone Project

#### Project Summary
The Organization for Tourism Development (OTD) want to analyze migration flux in USA, in order to find insights to significantly and sustainably develop the tourism in USA. To support their core idea they have identified a set of analysis/queries they want to run on the raw data available. The project deals with building a ETL data pipeline, to go from raw data to the data insights on the migration flux.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import glob
import os
import logging

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, FloatType, TimestampType  

### Step 1: Scope the Project and Gather Data

#### Scope 
##### What data 
This project using 4 datasets includes:
* I94 immigration data for year 2016. Used for the main analysis
* World Temperature Data
* Airport Code Table
* U.S. City Demographic Data

##### What tools
* Apache Hadoop: using to stored data
* Apche Spark: using to transform and analyst data
* Apache Aiflow: using to schedule tasks

#### Describe and Gather Data 

[I94 Immigration Data](https://travel.trade.gov/research/reports/i94/historical/2016.html) (SAS format): The data are provided by the US National Tourism and Trade Office. It is a collection of all I94 that have been filed in 2016.

[World Temperature Data](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data) (CSV format): This dataset is from Kaggle and contains monthly average temperature data at different country in the world wide.

[U.S. City Demographic Data](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/) (CSV format): The dataset comes from OpenSoft. It contains demographic info on US cities. The info are organized like in the picture below.

[Airport Codes Data](https://datahub.io/core/airport-codes#data) (CSV format)
This is a table of airport codes, and information on the corresponding cities, like gps coordinates, elevation, country, etc.

In [17]:
# Read sample immigration data using pandas
immi_df = pd.read_csv("immigration_data_sample.csv").iloc[:,1:]

In [18]:
immi_df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [20]:
immi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
cicid       1000 non-null float64
i94yr       1000 non-null float64
i94mon      1000 non-null float64
i94cit      1000 non-null float64
i94res      1000 non-null float64
i94port     1000 non-null object
arrdate     1000 non-null float64
i94mode     1000 non-null float64
i94addr     941 non-null object
depdate     951 non-null float64
i94bir      1000 non-null float64
i94visa     1000 non-null float64
count       1000 non-null float64
dtadfile    1000 non-null int64
visapost    382 non-null object
occup       4 non-null object
entdepa     1000 non-null object
entdepd     954 non-null object
entdepu     0 non-null float64
matflag     954 non-null object
biryear     1000 non-null float64
dtaddto     1000 non-null object
gender      859 non-null object
insnum      35 non-null float64
airline     967 non-null object
admnum      1000 non-null float64
fltno       992 non-null object
visat

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

In [3]:
# Read a immigration sas data file using spark
immi_spark_df = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [23]:
immi_spark_df.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [24]:
immi_spark_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [25]:
immi_spark_df.count()

3096313

In [11]:
# Write spark dataframe to parquet files
immi_spark_df.write.parquet("sas_data")
immi_spark_df=spark.read.parquet("sas_data")

In [36]:
# Read temperature data to dataframe
temp_df = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')
temp_df.info()
temp_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [34]:
# Read temperature data to dataframe
demo_df = pd.read_csv('us-cities-demographics.csv', delimiter=';')
demo_df.info()
demo_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [32]:
 # Read airport data to dataframe
airport_df = pd.read_csv('airport-codes_csv.csv')
airport_df.info()
airport_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


### Step 2: Explore and Assess the Data
#### Explore the Data 
1. Use pandas for exploratory data analysis to get an overview on these data sets (above)
2. Use PySpark for spliting data sets to dimensional tables and change column names for better understanding

#### Immigration data

In [4]:
# Read immigration data file to dataframe
immi_spark_df = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [5]:
# Change data type of some columns from double to integer
toInt = udf(lambda x: int(x) if x!=None else x, IntegerType())

for colname, coltype in immi_spark_df.dtypes:
    if coltype == 'double':
        immi_spark_df = immi_spark_df.withColumn(colname, toInt(colname))

In [6]:
immi_spark_df.printSchema()

root
 |-- cicid: integer (nullable = true)
 |-- i94yr: integer (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: integer (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: integer (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: integer

In [7]:
# Extract columns to create fact_immigration table
fact_immigration_df = immi_spark_df.select('cicid', 'i94port', 'i94addr', 'i94visa', 'i94yr', \
                                'i94mon', 'i94mode', 'arrdate', 'depdate').distinct()

In [8]:
# Rename columns of fact_immigration table
fact_immigration_df = fact_immigration_df.withColumnRenamed('i94port', 'city_code') \
                                .withColumnRenamed('i94addr', 'state_code') \
                                .withColumnRenamed('i94visa', 'visa') \
                                .withColumnRenamed('i94yr', 'year') \
                                .withColumnRenamed('i94mon', 'month') \
                                .withColumnRenamed('i94mode', 'mode') \
                                .withColumnRenamed('arrdate', 'arrive_date') \
                                .withColumnRenamed('depdate', 'departure_date')

In [9]:
fact_immigration_df.count()

3096313

In [10]:
# Drop null records on state_code column
fact_immigration_df = fact_immigration_df.where(col('state_code').isNotNull())

In [51]:
fact_immigration_df.count()

2943721

In [11]:
# Add country column to fact_immigration table
fact_immigration_df = fact_immigration_df.withColumn('country', lit('United States'))

In [12]:
# Change date type from SAS to timestamp
@udf(TimestampType())
def to_timestamp_udf(x):
    try:
        return pd.to_timedelta(x, unit='D') + pd.Timestamp('1960-1-1')
    except:
        return pd.Timestamp('1900-1-1')
    
fact_immigration_df = fact_immigration_df.withColumn('arrive_date', to_date(to_timestamp_udf(col('arrive_date')))) \
                                    .withColumn('departure_date', to_date(to_timestamp_udf(col('departure_date'))))

In [79]:
fact_immigration_df.printSchema()
fact_immigration_df.show(5)

root
 |-- cicid: integer (nullable = true)
 |-- city_code: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- visa: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- mode: integer (nullable = true)
 |-- arrive_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- country: string (nullable = false)

+-----+---------+----------+----+----+-----+----+-----------+--------------+-------------+
|cicid|city_code|state_code|visa|year|month|mode|arrive_date|departure_date|      country|
+-----+---------+----------+----+----+-----+----+-----------+--------------+-------------+
|  814|      ATL|        GA|   1|2016|    4|   1| 2016-04-01|    2016-04-02|United States|
| 1158|      WAS|        MD|   2|2016|    4|   1| 2016-04-01|    2016-04-03|United States|
| 1475|      NYC|        NY|   2|2016|    4|   1| 2016-04-01|    2016-04-08|United States|
| 1581|      LOS|        CA|   2|2016|    4|   1| 201

In [13]:
# Extract columns to create dim_immigrate_person
dim_immigrate_person_df = immi_spark_df.select('cicid', 'i94cit', 'i94res',\
                                        'biryear', 'gender', 'insnum').distinct()

In [14]:
# Rename columns of dim_immigrate_person table
dim_immigrate_person_df = dim_immigrate_person_df.withColumnRenamed('i94cit', 'citizen_country_code') \
                                    .withColumnRenamed('i94res', 'citizen_state_code') \
                                    .withColumnRenamed('biryear', 'birth_year')

In [77]:
dim_immigrate_person_df.printSchema()
dim_immigrate_person_df.show(5)

root
 |-- cicid: integer (nullable = true)
 |-- citizen_country_code: integer (nullable = true)
 |-- citizen_state_code: integer (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)

+-----+--------------------+------------------+----------+------+------+
|cicid|citizen_country_code|citizen_state_code|birth_year|gender|insnum|
+-----+--------------------+------------------+----------+------+------+
|  286|                 103|               103|      1992|     F|  null|
|  763|                 103|               103|      1975|  null|  null|
|  877|                 104|               104|      1975|     F|  null|
| 1336|                 104|               104|      1961|     M|  null|
| 2307|                 107|               107|      1961|     M|  null|
+-----+--------------------+------------------+----------+------+------+
only showing top 5 rows



#### Temperature Data

In [16]:
# Read temperature data file to dataframe using spark
temp_df = spark.read.csv('../../data2/GlobalLandTemperaturesByCity.csv', header=True)

In [17]:
# Filter data in United States
temp_df = temp_df.where(temp_df['Country'] == 'United States')

In [18]:
# Extract columns to create dim_temperature table
dim_temperature_df = temp_df.select('dt', 'AverageTemperature', 'AverageTemperatureUncertainty', \
                            'City', 'Country').distinct()

In [19]:
# Rename columns of dim_temperature
dim_temperature_df = dim_temperature_df.withColumnRenamed('AverageTemperature', 'avg_temperture') \
                                .withColumnRenamed('AverageTemperatureUncertainty', 'avg_temp_uncertainty') \
                                .withColumnRenamed('City', 'city') \
                                .withColumnRenamed('Country', 'country')

In [78]:
dim_temperature_df.printSchema()
dim_temperature_df.show(5)

root
 |-- dt: string (nullable = true)
 |-- avg_temperture: string (nullable = true)
 |-- avg_temp_uncertainty: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)

+----------+------------------+--------------------+-------+-------------+
|        dt|    avg_temperture|avg_temp_uncertainty|   city|      country|
+----------+------------------+--------------------+-------+-------------+
|1821-06-01|            25.768|               2.653|Abilene|United States|
|1830-11-01|            13.302|               2.715|Abilene|United States|
|1836-11-01|             8.827|  2.1719999999999997|Abilene|United States|
|1846-07-01|28.258000000000003|  2.0069999999999997|Abilene|United States|
|1863-04-01|            18.553|  1.1740000000000002|Abilene|United States|
+----------+------------------+--------------------+-------+-------------+
only showing top 5 rows



#### Demographic data

In [20]:
# Read demographic data to dataframe using spark
demo_df = spark.read.format('csv').options(header=True, delimiter=';').load('us-cities-demographics.csv')

In [21]:
# Rename columns of dim_demographic table
dim_demographic_df  = demo_df.withColumnRenamed('City','city') \
                                .withColumnRenamed('State','state') \
                                .withColumnRenamed('Median Age','median_age') \
                                .withColumnRenamed('Male Population','male_population') \
                                .withColumnRenamed('Female Population','female_population') \
                                .withColumnRenamed('Total Population','total_population') \
                                .withColumnRenamed('Number of Veterans','number_of_veterans') \
                                .withColumnRenamed('Foreign-born','foreign_born') \
                                .withColumnRenamed('Average Household Size','avg_household_size') \
                                .withColumnRenamed('State Code','state_code')\
                                .withColumnRenamed('Race','race') \
                                .withColumnRenamed('Count','count')

In [88]:
dim_demographic_df.printSchema()
dim_demographic_df.show(5)

root
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- median_age: string (nullable = true)
 |-- male_population: string (nullable = true)
 |-- female_population: string (nullable = true)
 |-- total_population: string (nullable = true)
 |-- number_of_veterans: string (nullable = true)
 |-- foreign_born: string (nullable = true)
 |-- avg_household_size: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- race: string (nullable = true)
 |-- count: string (nullable = true)

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+------------------+----------+--------------------+-----+
|            city|        state|median_age|male_population|female_population|total_population|number_of_veterans|foreign_born|avg_household_size|state_code|                race|count|
+----------------+-------------+----------+---------------+-----------------+----------------+--------------

In [22]:
# Change type of some columns to float
colnames = ['median_age', 'male_population', 'female_population', 'total_population', 
            'number_of_veterans', 'foreign_born', 'avg_household_size', 'count']

toFloat = udf(lambda x: float(x) if x!=None else x, FloatType())

for colname in colnames:
    dim_demographic_df = dim_demographic_df.withColumn(colname, toFloat(colname))

In [93]:
dim_demographic_df.printSchema()
dim_demographic_df.show(5)

root
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- median_age: float (nullable = true)
 |-- male_population: float (nullable = true)
 |-- female_population: float (nullable = true)
 |-- total_population: float (nullable = true)
 |-- number_of_veterans: float (nullable = true)
 |-- foreign_born: float (nullable = true)
 |-- avg_household_size: float (nullable = true)
 |-- state_code: string (nullable = true)
 |-- race: string (nullable = true)
 |-- count: float (nullable = true)

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+------------------+----------+--------------------+-------+
|            city|        state|median_age|male_population|female_population|total_population|number_of_veterans|foreign_born|avg_household_size|state_code|                race|  count|
+----------------+-------------+----------+---------------+-----------------+----------------+------------------

#### Airport data

In [23]:
# Read airport data to dataframe using spark
airport_df = spark.read.csv('airport-codes_csv.csv', header=True)

In [24]:
# Convert elevation_ft to type integer
airport_df = airport_df.withColumn('elevation_ft', col('elevation_ft').cast('integer'))

In [25]:
# Add state_code column to join with fact table
airport_df = airport_df.withColumn('state_code', split(col('iso_region'), '-').getItem(1)) 

In [98]:
airport_df.printSchema()
airport_df.show(5)

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- state_code: string (nullable = true)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+----------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|state_code|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+---

In [26]:
# Extract columns to create dim_airport table
dim_airport_df = airport_df.select('ident', 'type', 'name', 'elevation_ft','iso_country', \
                                'state_code', 'municipality', 'coordinates')

In [101]:
dim_airport_df.printSchema()
dim_airport_df.show(5)

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- coordinates: string (nullable = true)

+-----+-------------+--------------------+------------+-----------+----------+------------+--------------------+
|ident|         type|                name|elevation_ft|iso_country|state_code|municipality|         coordinates|
+-----+-------------+--------------------+------------+-----------+----------+------------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|         US|        PA|    Bensalem|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|         US|        KS|       Leoti|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|         US|        AK|Anchor Point|-151.6959991

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Star schema
![diagram](concept_model.png)

#### 3.2 Mapping Out Data Pipelines
Fact Immigration:
- Read immigration data file to dataframe
- Change data type of some columns from double to integer
- Extract columns to create fact_immigration table
- Rename columns of fact_immigration table
- Drop null records on state_code column
- Add country column to fact_immigration table
- Change date type from SAS to timestamp
- Write fact_immigration table to parquet files and partition by state_code

Dim Immigrate Person:
- Extract columns to create dim_immigrate_person from immigration data
- Rename columns of dim_immigrate_person table
- Write dim_immigrat_person table to parquet files  

Dim Temperature
- Read temperature data file to dataframe
- Filter data in United States
- Extract columns to create dim_temperature table
- Rename columns of dim_temperature
- Extract year, month from dt column
- Write dim_temperature to parquet files

Dim Demographic
- Read demographic data to dataframe
- Rename columns of dim_demographic table
- Change type of some columns to float
- Write dim_demographic table to parquet files

Dim Airport
- Read airport data to dataframe
- Convert elevation_ft to type integer
- Add state_code column to join with fact table
- Extract columns to create dim_airport table
- Write dim_airport table to parquet files

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

fact_immigration_df = immi_spark_df.select('cicid', 'i94port', 'i94addr', 'i94visa', 'i94yr', \
                                'i94mon', 'i94mode', 'arrdate', 'depdate').distinct()
fact_immigration_df = fact_immigration_df.withColumnRenamed('i94port', 'city_code') \
                                .withColumnRenamed('i94addr', 'state_code') \
                                .withColumnRenamed('i94visa', 'visa') \
                                .withColumnRenamed('i94yr', 'year') \
                                .withColumnRenamed('i94mon', 'month') \
                                .withColumnRenamed('i94mode', 'mode') \
                                .withColumnRenamed('arrdate', 'arrive_date') \
                                .withColumnRenamed('depdate', 'departure_date')

In [103]:
fact_immigration_df.show(5)

+-----+---------+----------+----+----+-----+----+-----------+--------------+-------------+
|cicid|city_code|state_code|visa|year|month|mode|arrive_date|departure_date|      country|
+-----+---------+----------+----+----+-----+----+-----------+--------------+-------------+
|  814|      ATL|        GA|   1|2016|    4|   1| 2016-04-01|    2016-04-02|United States|
| 1158|      WAS|        MD|   2|2016|    4|   1| 2016-04-01|    2016-04-03|United States|
| 1475|      NYC|        NY|   2|2016|    4|   1| 2016-04-01|    2016-04-08|United States|
| 1581|      LOS|        CA|   2|2016|    4|   1| 2016-04-01|    2016-04-10|United States|
| 1676|      MIA|        FL|   2|2016|    4|   1| 2016-04-01|    2016-04-21|United States|
+-----+---------+----------+----+----+-----+----+-----------+--------------+-------------+
only showing top 5 rows



dim_immigrate_person_df = immi_spark_df.select('cicid', 'i94cit', 'i94res',\
                                    'biryear', 'gender', 'insnum').distinct()
dim_immigrate_person_df = dim_immigrate_person_df.withColumnRenamed('i94cit', 'citizen_country_code') \
                                    .withColumnRenamed('i94res', 'citizen_state_code') \
                                    .withColumnRenamed('biryear', 'birth_year')

In [104]:
dim_immigrate_person_df.show(5)

+-----+--------------------+------------------+----------+------+------+
|cicid|citizen_country_code|citizen_state_code|birth_year|gender|insnum|
+-----+--------------------+------------------+----------+------+------+
|  286|                 103|               103|      1992|     F|  null|
|  763|                 103|               103|      1975|  null|  null|
|  877|                 104|               104|      1975|     F|  null|
| 1336|                 104|               104|      1961|     M|  null|
| 2307|                 107|               107|      1961|     M|  null|
+-----+--------------------+------------------+----------+------+------+
only showing top 5 rows



dim_temperature_df = temp_df.select('dt', 'AverageTemperature', 'AverageTemperatureUncertainty', \
                            'City', 'Country').distinct()
dim_temperature_df = dim_temperature_df.withColumnRenamed('AverageTemperature', 'avg_temperture') \
                                .withColumnRenamed('AverageTemperatureUncertainty', 'avg_temp_uncertainty') \
                                .withColumnRenamed('City', 'city') \
                                .withColumnRenamed('Country', 'country')

In [105]:
dim_temperature_df.show(5)

+----------+------------------+--------------------+-------+-------------+
|        dt|    avg_temperture|avg_temp_uncertainty|   city|      country|
+----------+------------------+--------------------+-------+-------------+
|1821-06-01|            25.768|               2.653|Abilene|United States|
|1830-11-01|            13.302|               2.715|Abilene|United States|
|1836-11-01|             8.827|  2.1719999999999997|Abilene|United States|
|1846-07-01|28.258000000000003|  2.0069999999999997|Abilene|United States|
|1863-04-01|            18.553|  1.1740000000000002|Abilene|United States|
+----------+------------------+--------------------+-------+-------------+
only showing top 5 rows



dim_demographic_df  = demo_df.withColumnRenamed('City','city') \
                                .withColumnRenamed('State','state') \
                                .withColumnRenamed('Median Age','median_age') \
                                .withColumnRenamed('Male Population','male_population') \
                                .withColumnRenamed('Female Population','female_population') \
                                .withColumnRenamed('Total Population','total_population') \
                                .withColumnRenamed('Number of Veterans','number_of_veterans') \
                                .withColumnRenamed('Foreign-born','foreign_born') \
                                .withColumnRenamed('Average Household Size','avg_household_size') \
                                .withColumnRenamed('State Code','state_code')\
                                .withColumnRenamed('Race','race') \
                                .withColumnRenamed('Count','count')

In [106]:
dim_demographic_df.show(5)

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+------------------+----------+--------------------+-------+
|            city|        state|median_age|male_population|female_population|total_population|number_of_veterans|foreign_born|avg_household_size|state_code|                race|  count|
+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+------------------+----------+--------------------+-------+
|   Silver Spring|     Maryland|      33.8|        40601.0|          41862.0|         82463.0|            1562.0|     30908.0|               2.6|        MD|  Hispanic or Latino|25924.0|
|          Quincy|Massachusetts|      41.0|        44129.0|          49500.0|         93629.0|            4147.0|     32935.0|              2.39|        MA|               White|58723.0|
|          Hoover|      Alabama|      38.5|        38040.0|          4

airport_df = airport_df.withColumn('state_code', split(col('iso_region'), '-').getItem(1)) 
dim_airport_df = airport_df.select('ident', 'type', 'name', 'elevation_ft','iso_country', \
                            'state_code', 'municipality', 'coordinates')

In [107]:
dim_airport_df.show(5)

+-----+-------------+--------------------+------------+-----------+----------+------------+--------------------+
|ident|         type|                name|elevation_ft|iso_country|state_code|municipality|         coordinates|
+-----+-------------+--------------------+------------+-----------+----------+------------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|         US|        PA|    Bensalem|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|         US|        KS|       Leoti|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|         US|        AK|Anchor Point|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|         US|        AL|     Harvest|-86.7703018188476...|
| 00AR|       closed|Newport Hospital ...|         237|         US|        AR|     Newport| -91.254898, 35.6087|
+-----+-------------+--------------------+------------+-----------+----------+------------+-----

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [27]:
# Create view for quality check
fact_immigration_df.createOrReplaceTempView('fact_immigration')
dim_immigrate_person_df.createOrReplaceTempView('dim_immigrate_person')
dim_temperature_df.createOrReplaceTempView('dim_temperature')
dim_demographic_df.createOrReplaceTempView('dim_demographic')
dim_airport_df.createOrReplaceTempView('dim_airport')

In [110]:
# Data Quality Check 01
tables = ['fact_immigration', 'dim_immigrate_person', 'dim_temperature', 'dim_demographic', 'dim_airport']

for table in tables:
    print(f'Checking data quality on table {table}...')
    result = spark.sql(f'''SELECT COUNT(*) FROM {table}''')
    if result.head()[0] > 0:
        print(f'Table {table} passed. {result.head()[0]} records')
    else:
        print(f'Table {table}. Data quality check failed!')

Checking data quality on table fact_immigration...
Table fact_immigration passed. 2943721 records
Checking data quality on table dim_immigrate_person...
Table dim_immigrate_person passed. 3096313 records
Checking data quality on table dim_temperature...
Table dim_temperature passed. 687004 records
Checking data quality on table dim_demographic...
Table dim_demographic passed. 2891 records
Checking data quality on table dim_airport...
Table dim_airport passed. 55075 records


In [33]:
# Data Quality Check 02
tables = ['fact_immigration', 'dim_demographic', 'dim_airport']

for table in tables:
    print(f"Checking data quality on table {table}...")
    expected_result = spark.sql(f"""SELECT COUNT(*) FROM {table} WHERE state_code IS NULL""")
    if expected_result.head()[0] > 0:
        print(f"Table {table}.  Data quality check failed! Found NULL state_code values in {table} table!")
    else:
        print(f"Table {table} check passed")

# For dim_immigrate_person and dim_temperature data quality check
print(f"Checking data quality on table dim_temperature...")
expected_result = spark.sql(f"""SELECT COUNT(*) FROM dim_temperature WHERE city = 'NY'""")
if expected_result.head()[0] < 0:
    print(f"Table {table}.  Data quality check failed! Cannot found NY city in dim_temperature table!")
else:
    print(f"Table {table} check passed")
    
print(f"Checking data quality on table dim_immigrate_person...")
expected_result = spark.sql(f"""SELECT COUNT(*) FROM dim_immigrate_person WHERE citizen_state_code IS NULL""")
if expected_result.head()[0] > 0:
    print(f"Table dim_immigrate_person.  Data quality check failed! Found NULL citizen_state_code in dim_immigrate_person!")
else:
    print(f"Table dim_immigrate_person check passed")

Checking data quality on table fact_immigration...
Table fact_immigration check passed
Checking data quality on table dim_demographic...
Table dim_demographic check passed
Checking data quality on table dim_airport...
Table dim_airport check passed
Checking data quality on table dim_temperature...
Table dim_airport check passed
Checking data quality on table dim_immigrate_person...
Table dim_immigrate_person check passed


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where 


In [112]:
fact_immigration
 |-- cicid: integer (nullable = true)
 |-- city_code: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- visa: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- mode: integer (nullable = true)
 |-- arrive_date: date (nullable = true)
 |-- departure_date: date (nullable = true)
 |-- country: string (nullable = false)

dim_immigrate_person
 |-- cicid: integer (nullable = true)
 |-- citizen_country_code: integer (nullable = true)
 |-- citizen_state_code: integer (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)

dim_temperature
 |-- dt: string (nullable = true)
 |-- avg_temperture: string (nullable = true)
 |-- avg_temp_uncertainty: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)

dim_demographic
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- median_age: float (nullable = true)
 |-- male_population: float (nullable = true)
 |-- female_population: float (nullable = true)
 |-- total_population: float (nullable = true)
 |-- number_of_veterans: float (nullable = true)
 |-- foreign_born: float (nullable = true)
 |-- avg_household_size: float (nullable = true)
 |-- state_code: string (nullable = true)
 |-- race: string (nullable = true)
 |-- count: float (nullable = true)


dim_airport
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- coordinates: string (nullable = true) 

IndentationError: unexpected indent (<ipython-input-112-e49895fd66bb>, line 2)

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
    - Apache Spark:
        + Speed
        + Ease of Use
        + Advanced Analytics
        + Dynamic in Nature
        + Multilingual
        + Apache Spark is powerful
        + Increased access to Big data
        + Open-source community
    - Apache Airflow:
        + Ease of use
        + Integrations
        + Coding with standard Python
        + Graphical UI
* Propose how often the data should be updated and why.
    - Data should be updated monthly because the Immigration data is updated monthly
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
     - In the local machine, we can use the Spark cluster and Hadoop to scale 
     - Or we can migarte the data into the cloud like AWS with ERM cluster and S3
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
     - Using Apache Airflow is one of the best solution to schedule the data pipeline
 * The database needed to be accessed by 100+ people.
     - AWS Redshift can handle up to 500 connections and we totaly use the Redshift database for a lot of users.

###  Some sample questions and sample queries

What city registered the highest number of arrivals?

In [38]:
fact_immigration_df.groupBy('city_code').count().sort(col('count').desc()).persist().show(10)

+---------+------+
|city_code| count|
+---------+------+
|      NYC|474904|
|      MIA|329385|
|      LOS|292243|
|      SFR|148637|
|      ORL|144693|
|      HHW|137100|
|      NEW|133376|
|      CHI|126139|
|      HOU| 95736|
|      FTL| 91756|
+---------+------+
only showing top 10 rows



The gender distribution of the applicants

In [35]:
dim_immigrate_person_df.select("gender").groupBy("gender").count().show()

+------+-------+
|gender|  count|
+------+-------+
|     F|1302743|
|  null| 414269|
|     M|1377224|
|     U|    467|
|     X|   1610|
+------+-------+



What city has the most population?

In [42]:
dim_demographic_df.groupBy('city').max('total_population').orderBy(col('max(total_population)').desc()).show(5)

+------------+---------------------+
|        city|max(total_population)|
+------------+---------------------+
|    New York|            8550405.0|
| Los Angeles|            3971896.0|
|     Chicago|            2720556.0|
|     Houston|            2298628.0|
|Philadelphia|            1567442.0|
+------------+---------------------+
only showing top 5 rows



What state has the most veterans?

In [47]:
dim_demographic_df.groupBy('state').sum('number_of_veterans').orderBy(col('sum(number_of_veterans)').desc()).show(5)

+----------+-----------------------+
|     state|sum(number_of_veterans)|
+----------+-----------------------+
|California|              4617022.0|
|     Texas|              3429512.0|
|   Florida|              1861951.0|
|   Arizona|              1322525.0|
|  Virginia|              1148830.0|
+----------+-----------------------+
only showing top 5 rows



What is the max and min average temperature in US?

In [51]:
dim_temperature_df.select(max('avg_temperture')).show()
dim_temperature_df.select(min('avg_temperture')).show()

+-------------------+
|max(avg_temperture)|
+-------------------+
|              9.999|
+-------------------+

+--------------------+
| min(avg_temperture)|
+--------------------+
|-0.00099999999999989|
+--------------------+



In [53]:
# Number of airports by each type
dim_airport_df.groupBy("type").count().distinct().show()

+--------------+-----+
|          type|count|
+--------------+-----+
| large_airport|  627|
|   balloonport|   24|
| seaplane_base| 1016|
|      heliport|11287|
|        closed| 3606|
|medium_airport| 4550|
| small_airport|33965|
+--------------+-----+



In [113]:
spark.stop()